In [3]:
import numpy as np
import pandas as pd
import os
import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt

In [4]:
os.getcwd()

'C:\\Users\\hamma\\Desktop\\WUST\\First Semester\\Business Intelligence Workplace\\Project 2\\Hammad_Ullah_P02'

In [12]:
day_ahead_2yr = pd.read_csv('Day-ahead_prices_201901010000_202101012359.csv', delimiter=';', usecols=(0,1,5))
day_ahead_rest = pd.read_csv('Day-ahead_prices_202101020000_202110252359.csv', delimiter=';', usecols=(0,1,5))
day_ahead_price_all = pd.concat([day_ahead_2yr, day_ahead_rest],ignore_index=True)
day_ahead_price_all.columns = ['Date', 'TimeOfDay', 'France___MWh_']


newRows = {'Date': ["Mar 31, 2019","Mar 29, 2020","Mar 28, 2021","Oct 27, 2019","Oct 25, 2020"],
        'TimeOfDay': ["2:00 AM","2:00 AM","2:00 AM","2:00 AM","2:00 AM"],
       'France___MWh_':[day_ahead_price_all.France___MWh_[2137:2139].mean(),
                        day_ahead_price_all.France___MWh_[10873:10875].mean(),
                       day_ahead_price_all.France___MWh_[19609:19611].mean(),
                        day_ahead_price_all.France___MWh_[7177:7179].mean(),
                       day_ahead_price_all.France___MWh_[15913:15915].mean()]}
newRows = pd.DataFrame.from_dict(newRows,orient='columns')


final_data_day_ahead = pd.concat([day_ahead_price_all[0:2138],newRows[0:1],
                                 day_ahead_price_all[2138:7177],newRows[3:4],
                                 day_ahead_price_all[7179:10874],newRows[1:2],
                                 day_ahead_price_all[10874:15913],newRows[4:5],
                                 day_ahead_price_all[15915:19610],newRows[2:3],
                                 day_ahead_price_all[19610::]],ignore_index=True)


final_day_ahead_demand_all = pd.read_csv('FRhourlyload.csv', delimiter=',',parse_dates=['Time (CET/CEST)'])
final_day_ahead_demand_all.columns = ['Date', 'DayAheadDemand']
data = pd.concat([final_day_ahead_demand_all['Date'],
                  final_data_day_ahead['France___MWh_'],
                  final_day_ahead_demand_all['DayAheadDemand']],axis=1,ignore_index=False)

days = [1,2,3,4,5,6,0]*1029
Weekday = pd.DataFrame(np.zeros((24696, 1)))
idx = 0
for i in days:
    Weekday[idx:idx+24] = i
    idx = idx + 24
data['Weekday'] = Weekday  

mincol = np.zeros(24720)
for i in range(24,24721,24):
    mincol[i:i+24] = data['France___MWh_'][i-24:i].min()
mincol = pd.DataFrame(mincol[24:24721])
data = pd.concat([data,mincol],axis=1,ignore_index=False)


data.columns = ['Date', 'DayAheadPrice', 'DayAheadDemand','Weekday','MinCol']


hr_24_prev_day = np.zeros(24720)
for i in range(24,24721,24):
    hr_24_prev_day[i:i+24] = data['DayAheadPrice'][i-1:i]
hr_24_prev_day = pd.DataFrame(hr_24_prev_day[24:24721])
data = pd.concat([data,hr_24_prev_day],axis=1,ignore_index=True)


data.columns = ['Date', 'DayAheadPrice', 'DayAheadDemand','Weekday','MinCol','24hrbefore']
data

,Date,DayAheadPrice,DayAheadDemand,Weekday,MinCol,24hrbefore
0,2019-01-01 00:00:00,51.00,63450.0,1.0,20.90,54.91
1,2019-01-01 01:00:00,46.27,60800.0,1.0,20.90,54.91
2,2019-01-01 02:00:00,39.78,59950.0,1.0,20.90,54.91
3,2019-01-01 03:00:00,27.87,56400.0,1.0,20.90,54.91
4,2019-01-01 04:00:00,23.21,53550.0,1.0,20.90,54.91
5,2019-01-01 05:00:00,22.64,52200.0,1.0,20.90,54.91
6,2019-01-01 06:00:00,20.90,52150.0,1.0,20.90,54.91
7,2019-01-01 07:00:00,26.54,52200.0,1.0,20.90,54.91
8,2019-01-01 08:00:00,25.00,52050.0,1.0,20.90,54.91
9,2019-01-01 09:00:00,25.00,52200.0,1.0,20.90,54.91


In [13]:
dumsat = np.zeros(24696)
dumsun = np.zeros(24696)
dummon = np.zeros(24696)

for i in range(24696):
    if data['Weekday'][i] == 5:
        dumsat[i] = 1
    if data['Weekday'][i] == 6:
        dumsun[i] = 1   
    if data['Weekday'][i] == 0:
        dummon[i] = 1
        
dumsat = pd.DataFrame(dumsat)
dumsun = pd.DataFrame(dumsun)
dummon = pd.DataFrame(dummon)

data = pd.concat([data,dumsat,dumsun,dummon],axis=1,ignore_index=False)


data.columns = ['Date', 'DayAheadPrice', 'DayAheadDemand','Weekday','MinCol','24hrbefore','DummSat','DummSun','DummMon']
data

,Date,DayAheadPrice,DayAheadDemand,Weekday,MinCol,24hrbefore,DummSat,DummSun,DummMon
0,2019-01-01 00:00:00,51.00,63450.0,1.0,20.90,54.91,0.0,0.0,0.0
1,2019-01-01 01:00:00,46.27,60800.0,1.0,20.90,54.91,0.0,0.0,0.0
2,2019-01-01 02:00:00,39.78,59950.0,1.0,20.90,54.91,0.0,0.0,0.0
3,2019-01-01 03:00:00,27.87,56400.0,1.0,20.90,54.91,0.0,0.0,0.0
4,2019-01-01 04:00:00,23.21,53550.0,1.0,20.90,54.91,0.0,0.0,0.0
5,2019-01-01 05:00:00,22.64,52200.0,1.0,20.90,54.91,0.0,0.0,0.0
6,2019-01-01 06:00:00,20.90,52150.0,1.0,20.90,54.91,0.0,0.0,0.0
7,2019-01-01 07:00:00,26.54,52200.0,1.0,20.90,54.91,0.0,0.0,0.0
8,2019-01-01 08:00:00,25.00,52050.0,1.0,20.90,54.91,0.0,0.0,0.0
9,2019-01-01 09:00:00,25.00,52200.0,1.0,20.90,54.91,0.0,0.0,0.0


In [14]:
data = data.to_numpy()
data = data[:, [1,2,3,4,5,6,7,8]]
cal_len = 731
test_len = data.shape[0] // 24 - cal_len
real = data[-298*24:, 0]
data = data.astype('float64')
data

array([[5.1000e+01, 6.3450e+04, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [4.6270e+01, 6.0800e+04, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.9780e+01, 5.9950e+04, 1.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [2.2423e+02, 5.3800e+04, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [2.1963e+02, 5.1750e+04, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00],
       [2.0547e+02, 5.4000e+04, 0.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

In [5]:
import tensorflow
from tensorflow import keras
import numpy as np
import optuna # !!!
import logging
import sys
cal_len = 731
test_len = data.shape[0] // 24 - cal_len
real = data[-298*24:, 0].reshape((298, 24))
price = data[:731*24, 0].reshape((731, 24))
load = data[:731*24, 1].reshape((731, 24))
#dumm = data[:731*24:, 2].reshape((731, 24))
minn = data[:731*24, 3].reshape((731, 24))
prev24 = data[:731*24,4].reshape((731, 24))
dumsat = data[:731*24, 5].reshape((731, 24))
dumsun = data[:731*24, 6].reshape((731, 24))
dummon = data[:731*24, 7].reshape((731, 24))


real_train = data[:731*24, 0]

y = real_train.reshape((731, 24))
y = y[7:]

x = np.hstack([price[6:-1, :], price[:-7, :], load[7:, :], minn[6:-1, 0:1],prev24[6:-1, 0:1],
              dumsat[7:, 0:1], dumsun[7:, 0:1],dummon[7:, 0:1]])

#x= x.T
#x = x.reshape((724, 24*9))

pricef = data[:, 0].reshape((1029, 24))
loadf = data[:, 1].reshape((1029, 24))
minnf = data[:, 3].reshape((1029, 24))
prev24f = data[:,4].reshape((1029, 24))
dumsatf = data[:, 5].reshape((1029, 24))
dumsunf = data[:, 6].reshape((1029, 24))
dummonf = data[:, 7].reshape((1029, 24))

xf = np.hstack([pricef[730:-1, :], pricef[724:-7, :], loadf[731:, :],
                   minnf[730:-1, 0:1], prev24f[731:, 0:1],
                  dumsatf[731:, 0:1], dumsunf[731:, 0:1], dummonf[731:, 0:1]])
#xf = xf.T
#xf = xf.reshape((298, 24*9))


permutation = np.random.permutation(list(range(724)))
percentage_val = 0.2
Nval = int(724 * percentage_val)
seen_x = x[permutation[:724-Nval], :]
seen_y = y[permutation[:724-Nval], :]
unseen_x = x[permutation[724-Nval:], :]
unseen_y = y[permutation[724-Nval:], :]






C:\Users\hamma\Anaconda3\envs\tensorflow\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'data' is not defined

In [ ]:
price[6:-1, :].shape

In [22]:
def objective(trial):
    inputs = keras.Input((77)) # Input layer of size equal to number of columns
    norm = keras.layers.BatchNormalization()(inputs)
    hidden = keras.layers.Dense(trial.suggest_int('hidden_neurons', 100, 500), activation='sigmoid')(norm)
    hidden2 = keras.layers.Dense(trial.suggest_int('hidden_neurons2', 100, 500), activation = 'sigmoid')(hidden)
    output = keras.layers.Dense(24, activation='linear')(hidden2)
    model = keras.Model(inputs=inputs, outputs=output)
    model.compile(optimizer=keras.optimizers.Adam(), loss='mse')
    model.fit(seen_x, seen_y, epochs=50, verbose=0, batch_size=32)
    pred = model.predict(unseen_x)
    return (np.mean(np.abs(pred - unseen_y)))

In [23]:
optuna.logging.get_logger('optuna').addHandler(logging.StreamHandler(sys.stdout))
study_name = 'TEST'
storage_name = f'sqlite:///trials/{study_name}'
study = optuna.create_study(study_name=study_name, storage=storage_name, load_if_exists=False)
study.optimize(objective, n_trials=10, show_progress_bar=True)
best_params = study.best_params
print(best_params)
print(study.trials_dataframe())

A new study created in RDB with name: TEST
A new study created in RDB with name: TEST


[I 2022-06-08 13:42:23,101] A new study created in RDB with name: TEST


A new study created in RDB with name: TEST
A new study created in RDB with name: TEST


C:\Users\hamma\Anaconda3\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


5/5 [==============================] - 0s 2ms/step
Trial 0 finished with value: 5.413518807170568 and parameters: {'hidden_neurons': 247, 'hidden_neurons2': 345}. Best is trial 0 with value: 5.413518807170568.
Trial 0 finished with value: 5.413518807170568 and parameters: {'hidden_neurons': 247, 'hidden_neurons2': 345}. Best is trial 0 with value: 5.413518807170568.
Trial 0 finished with value: 5.413518807170568 and parameters: {'hidden_neurons': 247, 'hidden_neurons2': 345}. Best is trial 0 with value: 5.413518807170568.
Trial 0 finished with value: 5.413518807170568 and parameters: {'hidden_neurons': 247, 'hidden_neurons2': 345}. Best is trial 0 with value: 5.413518807170568.
[I 2022-06-08 13:42:25,150] Trial 0 finished with value: 5.413518807170568 and parameters: {'hidden_neurons': 247, 'hidden_neurons2': 345}. Best is trial 0 with value: 5.413518807170568.
5/5 [==============================] - 0s 2ms/step
Trial 1 finished with value: 5.336936915439588 and parameters: {'hidden_neu

Trial 9 finished with value: 5.5998146854285835 and parameters: {'hidden_neurons': 165, 'hidden_neurons2': 399}. Best is trial 1 with value: 5.336936915439588.
Trial 9 finished with value: 5.5998146854285835 and parameters: {'hidden_neurons': 165, 'hidden_neurons2': 399}. Best is trial 1 with value: 5.336936915439588.
[I 2022-06-08 13:42:44,083] Trial 9 finished with value: 5.5998146854285835 and parameters: {'hidden_neurons': 165, 'hidden_neurons2': 399}. Best is trial 1 with value: 5.336936915439588.

{'hidden_neurons': 237, 'hidden_neurons2': 305}
   number     value             datetime_start          datetime_complete  \
0       0  5.413519 2022-06-08 13:42:23.153863 2022-06-08 13:42:25.087251   
1       1  5.336937 2022-06-08 13:42:25.166776 2022-06-08 13:42:26.931774   
2       2  5.504838 2022-06-08 13:42:27.017184 2022-06-08 13:42:29.150457   
3       3  6.265634 2022-06-08 13:42:29.216282 2022-06-08 13:42:30.702996   
4       4  5.740784 2022-06-08 13:42:30.780156 2022-06-08 

In [24]:
study.best_params

{'hidden_neurons': 237, 'hidden_neurons2': 305}

In [25]:
def evaluate(params):
    inputs = keras.Input((77))
    norm = keras.layers.BatchNormalization()(inputs)
    hidden = keras.layers.Dense(params['hidden_neurons'], activation="sigmoid")(norm)
    hidden2 = keras.layers.Dense(params['hidden_neurons2'], activation="sigmoid")(hidden)
    output = keras.layers.Dense(24, activation='linear')(hidden2)
    model = keras.Model(inputs=inputs, outputs=output)
    model.compile(optimizer=keras.optimizers.Adam(), loss='mse')
    model.fit(x, y, epochs=500, verbose=2, batch_size=32)
    preds = model.predict(xf)
    hr_analysis = []
    for hour in range(24):
        real_hour = real[hour::24]
        preds_hour = preds[hour::24]
        row = [(np.mean(np.abs(preds_hour - real_hour))), sqrt(mean_squared_error(real_hour, preds_hour)) ]
        hr_analysis.append(row)
    df = pd.DataFrame(hr_analysis)
    return(np.mean(np.abs(preds - real))), sqrt(mean_squared_error(real, preds)),df, preds

In [26]:
evaluate(study.best_params)

Epoch 1/500
23/23 - 0s - loss: 1319.2708 - 349ms/epoch - 15ms/step
Epoch 2/500
23/23 - 0s - loss: 950.3119 - 45ms/epoch - 2ms/step
Epoch 3/500
23/23 - 0s - loss: 647.6259 - 46ms/epoch - 2ms/step
Epoch 4/500
23/23 - 0s - loss: 433.8711 - 44ms/epoch - 2ms/step
Epoch 5/500
23/23 - 0s - loss: 304.1460 - 39ms/epoch - 2ms/step
Epoch 6/500
23/23 - 0s - loss: 223.0666 - 45ms/epoch - 2ms/step
Epoch 7/500
23/23 - 0s - loss: 175.1117 - 46ms/epoch - 2ms/step
Epoch 8/500
23/23 - 0s - loss: 146.9618 - 48ms/epoch - 2ms/step
Epoch 9/500
23/23 - 0s - loss: 131.1932 - 39ms/epoch - 2ms/step
Epoch 10/500
23/23 - 0s - loss: 121.0640 - 45ms/epoch - 2ms/step
Epoch 11/500
23/23 - 0s - loss: 112.6171 - 47ms/epoch - 2ms/step
Epoch 12/500
23/23 - 0s - loss: 106.4442 - 49ms/epoch - 2ms/step
Epoch 13/500
23/23 - 0s - loss: 102.0206 - 45ms/epoch - 2ms/step
Epoch 14/500
23/23 - 0s - loss: 99.1703 - 58ms/epoch - 3ms/step
Epoch 15/500
23/23 - 0s - loss: 94.8237 - 42ms/epoch - 2ms/step
Epoch 16/500
23/23 - 0s - loss: 9

Epoch 129/500
23/23 - 0s - loss: 39.5820 - 39ms/epoch - 2ms/step
Epoch 130/500
23/23 - 0s - loss: 38.4655 - 43ms/epoch - 2ms/step
Epoch 131/500
23/23 - 0s - loss: 39.2551 - 38ms/epoch - 2ms/step
Epoch 132/500
23/23 - 0s - loss: 38.5021 - 43ms/epoch - 2ms/step
Epoch 133/500
23/23 - 0s - loss: 38.3917 - 39ms/epoch - 2ms/step
Epoch 134/500
23/23 - 0s - loss: 40.1333 - 40ms/epoch - 2ms/step
Epoch 135/500
23/23 - 0s - loss: 41.2534 - 38ms/epoch - 2ms/step
Epoch 136/500
23/23 - 0s - loss: 36.9600 - 42ms/epoch - 2ms/step
Epoch 137/500
23/23 - 0s - loss: 38.0582 - 39ms/epoch - 2ms/step
Epoch 138/500
23/23 - 0s - loss: 37.9289 - 40ms/epoch - 2ms/step
Epoch 139/500
23/23 - 0s - loss: 37.4496 - 40ms/epoch - 2ms/step
Epoch 140/500
23/23 - 0s - loss: 38.6582 - 39ms/epoch - 2ms/step
Epoch 141/500
23/23 - 0s - loss: 35.8495 - 39ms/epoch - 2ms/step
Epoch 142/500
23/23 - 0s - loss: 36.4499 - 37ms/epoch - 2ms/step
Epoch 143/500
23/23 - 0s - loss: 36.9976 - 39ms/epoch - 2ms/step
Epoch 144/500
23/23 - 0s 

23/23 - 0s - loss: 28.4853 - 40ms/epoch - 2ms/step
Epoch 256/500
23/23 - 0s - loss: 30.3411 - 37ms/epoch - 2ms/step
Epoch 257/500
23/23 - 0s - loss: 32.7975 - 33ms/epoch - 1ms/step
Epoch 258/500
23/23 - 0s - loss: 29.9593 - 36ms/epoch - 2ms/step
Epoch 259/500
23/23 - 0s - loss: 29.8544 - 35ms/epoch - 2ms/step
Epoch 260/500
23/23 - 0s - loss: 28.8088 - 33ms/epoch - 1ms/step
Epoch 261/500
23/23 - 0s - loss: 31.7205 - 36ms/epoch - 2ms/step
Epoch 262/500
23/23 - 0s - loss: 33.0741 - 36ms/epoch - 2ms/step
Epoch 263/500
23/23 - 0s - loss: 32.1909 - 36ms/epoch - 2ms/step
Epoch 264/500
23/23 - 0s - loss: 29.0799 - 35ms/epoch - 2ms/step
Epoch 265/500
23/23 - 0s - loss: 28.7207 - 41ms/epoch - 2ms/step
Epoch 266/500
23/23 - 0s - loss: 31.1830 - 37ms/epoch - 2ms/step
Epoch 267/500
23/23 - 0s - loss: 29.3936 - 35ms/epoch - 2ms/step
Epoch 268/500
23/23 - 0s - loss: 30.8706 - 38ms/epoch - 2ms/step
Epoch 269/500
23/23 - 0s - loss: 31.7051 - 35ms/epoch - 2ms/step
Epoch 270/500
23/23 - 0s - loss: 31.096

Epoch 382/500
23/23 - 0s - loss: 23.8563 - 42ms/epoch - 2ms/step
Epoch 383/500
23/23 - 0s - loss: 27.4459 - 41ms/epoch - 2ms/step
Epoch 384/500
23/23 - 0s - loss: 24.2401 - 40ms/epoch - 2ms/step
Epoch 385/500
23/23 - 0s - loss: 25.6875 - 39ms/epoch - 2ms/step
Epoch 386/500
23/23 - 0s - loss: 25.8594 - 36ms/epoch - 2ms/step
Epoch 387/500
23/23 - 0s - loss: 22.8882 - 38ms/epoch - 2ms/step
Epoch 388/500
23/23 - 0s - loss: 28.8160 - 33ms/epoch - 1ms/step
Epoch 389/500
23/23 - 0s - loss: 23.3996 - 33ms/epoch - 1ms/step
Epoch 390/500
23/23 - 0s - loss: 25.7146 - 34ms/epoch - 1ms/step
Epoch 391/500
23/23 - 0s - loss: 24.8624 - 33ms/epoch - 1ms/step
Epoch 392/500
23/23 - 0s - loss: 24.9745 - 35ms/epoch - 2ms/step
Epoch 393/500
23/23 - 0s - loss: 24.1432 - 32ms/epoch - 1ms/step
Epoch 394/500
23/23 - 0s - loss: 24.7421 - 42ms/epoch - 2ms/step
Epoch 395/500
23/23 - 0s - loss: 25.3737 - 38ms/epoch - 2ms/step
Epoch 396/500
23/23 - 0s - loss: 24.2274 - 37ms/epoch - 2ms/step
Epoch 397/500
23/23 - 0s 

(30.430674230076324, 47.786695612213414,             0          1
 0   29.178650  47.316231
 1   29.900309  44.493058
 2   29.841944  47.816928
 3   29.334296  44.698895
 4   22.622422  33.901484
 5   29.770034  45.349738
 6   36.808228  55.958502
 7   35.398891  53.329596
 8   34.132838  50.399113
 9   31.794922  50.800090
 10  20.995557  29.137854
 11  20.966109  29.143989
 12  26.692328  40.700250
 13  28.285220  40.618792
 14  29.704046  46.933680
 15  40.772454  74.393851
 16  32.085340  50.683475
 17  31.559426  50.336078
 18  30.213575  46.022942
 19  31.247739  47.575756
 20  30.166395  47.156293
 21  27.897846  43.652989
 22  36.977508  52.681904
 23  33.617124  53.595559, array([[47.77606 , 44.513645, 42.494232, ..., 50.59841 , 51.075916,
         47.561306],
        [43.228046, 40.87071 , 38.952297, ..., 45.996563, 46.463272,
         43.648277],
        [42.686466, 40.520832, 38.72298 , ..., 48.837437, 48.446087,
         44.994022],
        ...,
        [55.519787, 51.2375